# Day 5: Advanced Architectures & Tool Integration in LangGraph

## 🎯 Learning Objectives
By the end of this session, you will:
- Build hierarchical multi-agent systems with complex organizational structures
- Implement parallel execution and map-reduce patterns for scalable processing
- Integrate external tools (Tavily search, APIs) with OpenAI for enhanced capabilities
- Optimize performance and reduce costs in LangGraph applications
- Deploy production-ready multi-agent systems
- Master advanced architectural patterns and best practices

## ⏱️ Session Structure (2 hours)
- **Learning Materials** (30 min): Advanced patterns and architecture theory
- **Hands-on Code** (60 min): Implementation and tool integration  
- **Practical Exercises** (30 min): Build scalable production systems

---

## 📖 Learning Materials (30 minutes)

### 📺 Video Resources
- [LangGraph Advanced Patterns](https://langchain-ai.github.io/langgraph/concepts/agentic_concepts/) - Agentic concepts and patterns
- [DeepLearning.AI - AI Agents in LangGraph](https://www.deeplearning.ai/short-courses/ai-agents-in-langgraph/) - Module 5: Advanced Architectures
- [LangChain Academy - Production Deployment](https://academy.langchain.com/) - Scaling and optimization

### 🧠 Theory: Advanced Architectural Patterns

#### Hierarchical Multi-Agent Systems
Hierarchical systems organize agents in layers with clear command structures:
- **Executive Layer**: High-level decision making and strategy
- **Management Layer**: Task coordination and resource allocation
- **Worker Layer**: Specialized task execution
- **Support Layer**: Shared services and utilities

#### Parallel Execution Patterns
1. **Map-Reduce**: Distribute work across multiple agents and aggregate results
2. **Pipeline Parallelism**: Different stages of processing run concurrently
3. **Data Parallelism**: Same operation on different data partitions
4. **Task Parallelism**: Different operations running simultaneously

#### Tool Integration Architecture
- **Tool Binding**: Direct integration with LangChain tools
- **API Orchestration**: Managing external service calls
- **Error Handling**: Resilient tool usage with fallbacks
- **Caching**: Optimizing repeated tool calls
- **Rate Limiting**: Respecting API constraints

#### Performance Optimization Strategies
1. **Model Selection**: Choosing appropriate models for different tasks
2. **Prompt Engineering**: Efficient prompts that reduce token usage
3. **Caching**: State and response caching
4. **Batching**: Grouping operations for efficiency
5. **Streaming**: Real-time response processing

#### Cost Optimization for OpenAI
- **Model Tiering**: Use GPT-3.5-turbo for simple tasks, GPT-4 for complex ones
- **Token Management**: Minimize input/output tokens
- **Response Filtering**: Stop generation early when possible
- **Batch Processing**: Combine multiple requests
- **Smart Retries**: Avoid unnecessary API calls

---
## 💻 Hands-on Code (60 minutes)

### Setup and Imports

In [ ]:
# Install required packages
!pip install langgraph langchain langchain-openai pydantic python-dotenv
!pip install langgraph-checkpoint-sqlite httpx aiohttp
!pip install tavily-python  # For web search integration
!pip install tiktoken  # For token counting
!pip install asyncio-throttle  # For rate limiting

In [ ]:
import os
import json
import asyncio
import time
from typing import TypedDict, Literal, List, Optional, Dict, Any, Union
from pydantic import BaseModel, Field, validator
from enum import Enum
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed
from dotenv import load_dotenv
import tiktoken

# LangGraph imports
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.types import Command
from langgraph.constants import INTERRUPT

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain_core.runnables import RunnableConfig

# Tool imports
from tavily import TavilyClient
import httpx

# Load environment variables
load_dotenv()

# Configure APIs
openai_api_key = os.getenv("OPENAI_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

if not openai_api_key:
    print("⚠️ Please set OPENAI_API_KEY in your .env file")
else:
    print("✅ OpenAI API key loaded successfully")

if not tavily_api_key:
    print("⚠️ Please set TAVILY_API_KEY in your .env file for web search")
    print("Get your key at: https://tavily.com/")
else:
    print("✅ Tavily API key loaded successfully")

# Initialize models with different configurations
executive_llm = ChatOpenAI(model="gpt-4", temperature=0.1, openai_api_key=openai_api_key)  # Strategic decisions
manager_llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key=openai_api_key)    # Planning and coordination
worker_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=openai_api_key)  # Task execution
fast_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, max_tokens=200, openai_api_key=openai_api_key)  # Quick tasks

# Initialize Tavily client
tavily_client = TavilyClient(api_key=tavily_api_key) if tavily_api_key else None

print("🚀 Advanced LangGraph setup completed")

### 1. Hierarchical Multi-Agent Architecture

In [ ]:
class AgentLevel(str, Enum):
    """Hierarchical levels in the organization"""
    EXECUTIVE = "executive"
    MANAGER = "manager"
    WORKER = "worker"
    SUPPORT = "support"

class TaskType(str, Enum):
    """Types of tasks in the system"""
    STRATEGIC = "strategic"
    OPERATIONAL = "operational"
    EXECUTION = "execution"
    ANALYSIS = "analysis"
    RESEARCH = "research"

class Task(BaseModel):
    """Task definition in hierarchical system"""
    task_id: str
    title: str
    description: str
    task_type: TaskType
    priority: int = Field(ge=1, le=5)  # 1 = highest priority
    assigned_level: AgentLevel
    assigned_agent: Optional[str] = None
    dependencies: List[str] = Field(default_factory=list)
    status: str = "pending"
    result: Optional[str] = None
    created_at: str = Field(default_factory=lambda: datetime.now().isoformat())
    completed_at: Optional[str] = None
    estimated_tokens: int = 0
    actual_tokens: int = 0

class HierarchicalState(BaseModel):
    """State for hierarchical multi-agent system"""
    messages: List[BaseMessage] = Field(default_factory=list)
    tasks: List[Task] = Field(default_factory=list)
    completed_tasks: List[Task] = Field(default_factory=list)
    current_task: Optional[Task] = None
    strategic_plan: Dict[str, Any] = Field(default_factory=dict)
    operational_plan: Dict[str, Any] = Field(default_factory=dict)
    execution_results: Dict[str, Any] = Field(default_factory=dict)
    agent_workload: Dict[str, int] = Field(default_factory=dict)
    performance_metrics: Dict[str, Any] = Field(default_factory=dict)
    total_tokens_used: int = 0
    estimated_cost: float = 0.0

# Token counting utility
def count_tokens(text: str, model: str = "gpt-3.5-turbo") -> int:
    """Count tokens in text for cost estimation"""
    try:
        encoding = tiktoken.encoding_for_model(model)
        return len(encoding.encode(text))
    except:
        # Fallback estimation
        return len(text) // 4

def estimate_cost(tokens: int, model: str = "gpt-3.5-turbo") -> float:
    """Estimate cost based on token usage"""
    # Current OpenAI pricing (approximate)
    pricing = {
        "gpt-3.5-turbo": 0.0015 / 1000,  # $0.0015 per 1K tokens
        "gpt-4": 0.03 / 1000,            # $0.03 per 1K tokens
    }
    rate = pricing.get(model, pricing["gpt-3.5-turbo"])
    return tokens * rate

# Executive layer - Strategic planning
def executive_agent(state: HierarchicalState) -> Command:
    """Executive agent handles strategic planning and high-level decisions"""
    print("👔 Executive: Developing strategic plan")
    
    # Analyze incoming tasks and create strategic plan
    pending_tasks = [t for t in state.tasks if t.status == "pending"]
    
    if pending_tasks:
        tasks_summary = "\n".join([f"- {t.title}: {t.description}" for t in pending_tasks])
        
        strategic_prompt = f"""
        You are the Executive Agent responsible for strategic planning.
        
        Analyze these pending tasks and create a strategic plan:
        {tasks_summary}
        
        Provide:
        1. Overall strategy and approach
        2. Priority ordering of tasks
        3. Resource allocation recommendations
        4. Risk assessment
        5. Success metrics
        
        Be concise but comprehensive.
        """
        
        # Count tokens for cost tracking
        input_tokens = count_tokens(strategic_prompt, "gpt-4")
        
        response = executive_llm.invoke([HumanMessage(content=strategic_prompt)])
        
        output_tokens = count_tokens(response.content, "gpt-4")
        total_tokens = input_tokens + output_tokens
        
        state.total_tokens_used += total_tokens
        state.estimated_cost += estimate_cost(total_tokens, "gpt-4")
        
        # Store strategic plan
        state.strategic_plan = {
            "plan": response.content,
            "created_at": datetime.now().isoformat(),
            "agent": "executive",
            "tokens_used": total_tokens
        }
        
        state.messages.append(response)
        
        print(f"📊 Executive: Strategic plan created (Tokens: {total_tokens}, Cost: ${state.estimated_cost:.4f})")
        
        return Command(goto="manager", update=state.model_dump())
    
    return Command(goto="manager", update=state.model_dump())

# Manager layer - Operational planning
def manager_agent(state: HierarchicalState) -> Command:
    """Manager agent creates operational plans and coordinates execution"""
    print("📋 Manager: Creating operational plan")
    
    pending_tasks = [t for t in state.tasks if t.status == "pending"]
    
    if pending_tasks and state.strategic_plan:
        operational_prompt = f"""
        You are the Manager Agent responsible for operational planning.
        
        Strategic Plan:
        {state.strategic_plan.get('plan', 'No strategic plan available')}
        
        Tasks to execute:
        {[t.title for t in pending_tasks]}
        
        Create an operational plan with:
        1. Task breakdown and sequencing
        2. Agent assignment recommendations
        3. Timeline and milestones
        4. Resource requirements
        5. Quality checkpoints
        
        Focus on practical execution steps.
        """
        
        input_tokens = count_tokens(operational_prompt, "gpt-4")
        response = manager_llm.invoke([HumanMessage(content=operational_prompt)])
        output_tokens = count_tokens(response.content, "gpt-4")
        total_tokens = input_tokens + output_tokens
        
        state.total_tokens_used += total_tokens
        state.estimated_cost += estimate_cost(total_tokens, "gpt-4")
        
        # Store operational plan
        state.operational_plan = {
            "plan": response.content,
            "created_at": datetime.now().isoformat(),
            "agent": "manager",
            "tokens_used": total_tokens
        }
        
        state.messages.append(response)
        
        # Assign tasks to workers based on operational plan
        for task in pending_tasks:
            if task.task_type in [TaskType.EXECUTION, TaskType.ANALYSIS, TaskType.RESEARCH]:
                task.assigned_level = AgentLevel.WORKER
                task.status = "assigned"
        
        print(f"📊 Manager: Operational plan created (Tokens: {total_tokens}, Cost: ${state.estimated_cost:.4f})")
        
        return Command(goto="worker_coordinator", update=state.model_dump())
    
    return Command(goto="worker_coordinator", update=state.model_dump())

# Worker coordinator - Manages parallel execution
def worker_coordinator(state: HierarchicalState) -> Command:
    """Coordinates parallel execution of worker tasks"""
    print("👥 Worker Coordinator: Orchestrating parallel execution")
    
    assigned_tasks = [t for t in state.tasks if t.status == "assigned"]
    
    if assigned_tasks:
        # Process tasks in parallel (simulated)
        for task in assigned_tasks:
            if task.task_type == TaskType.RESEARCH:
                task.assigned_agent = "research_worker"
            elif task.task_type == TaskType.ANALYSIS:
                task.assigned_agent = "analysis_worker"
            else:
                task.assigned_agent = "general_worker"
            
            task.status = "in_progress"
        
        print(f"⚡ Coordinator: {len(assigned_tasks)} tasks distributed to workers")
        
        # Start with first task (in real implementation, would be parallel)
        if assigned_tasks:
            state.current_task = assigned_tasks[0]
            
            if state.current_task.task_type == TaskType.RESEARCH:
                return Command(goto="research_worker", update=state.model_dump())
            elif state.current_task.task_type == TaskType.ANALYSIS:
                return Command(goto="analysis_worker", update=state.model_dump())
            else:
                return Command(goto="general_worker", update=state.model_dump())
    
    return Command(goto="results_aggregator", update=state.model_dump())

print("🏗️ Hierarchical agent architecture defined")

### 2. Tool Integration with Tavily and APIs

In [ ]:
# Tool integration utilities
class ToolResult(BaseModel):
    """Result from tool execution"""
    tool_name: str
    success: bool
    result: Any
    error: Optional[str] = None
    execution_time: float
    tokens_used: int = 0
    cost_estimate: float = 0.0

class ToolManager:
    """Manages tool integration and caching"""
    
    def __init__(self):
        self.cache = {}
        self.rate_limits = {
            "tavily": {"calls_per_minute": 100, "last_calls": []},
            "openai": {"calls_per_minute": 3000, "last_calls": []}
        }
    
    def _check_rate_limit(self, tool_name: str) -> bool:
        """Check if tool can be called without hitting rate limits"""
        if tool_name not in self.rate_limits:
            return True
        
        rate_info = self.rate_limits[tool_name]
        current_time = time.time()
        
        # Remove calls older than 1 minute
        rate_info["last_calls"] = [
            call_time for call_time in rate_info["last_calls"]
            if current_time - call_time < 60
        ]
        
        # Check if we can make another call
        return len(rate_info["last_calls"]) < rate_info["calls_per_minute"]
    
    def _record_call(self, tool_name: str):
        """Record a tool call for rate limiting"""
        if tool_name in self.rate_limits:
            self.rate_limits[tool_name]["last_calls"].append(time.time())
    
    def search_web(self, query: str, max_results: int = 5) -> ToolResult:
        """Search the web using Tavily"""
        start_time = time.time()
        
        # Check cache first
        cache_key = f"tavily_{query}_{max_results}"
        if cache_key in self.cache:
            print(f"🔍 Tavily: Using cached result for '{query}'")
            cached_result = self.cache[cache_key]
            cached_result.execution_time = time.time() - start_time
            return cached_result
        
        # Check rate limits
        if not self._check_rate_limit("tavily"):
            return ToolResult(
                tool_name="tavily",
                success=False,
                result=None,
                error="Rate limit exceeded",
                execution_time=time.time() - start_time
            )
        
        try:
            if not tavily_client:
                return ToolResult(
                    tool_name="tavily",
                    success=False,
                    result=None,
                    error="Tavily client not configured",
                    execution_time=time.time() - start_time
                )
            
            print(f"🔍 Tavily: Searching for '{query}'")
            
            # Perform search
            response = tavily_client.search(
                query=query,
                search_depth="basic",
                max_results=max_results
            )
            
            self._record_call("tavily")
            
            # Process results
            processed_results = []
            for result in response.get("results", []):
                processed_results.append({
                    "title": result.get("title", ""),
                    "url": result.get("url", ""),
                    "content": result.get("content", "")[:500],  # Limit content length
                    "score": result.get("score", 0)
                })
            
            result = ToolResult(
                tool_name="tavily",
                success=True,
                result=processed_results,
                execution_time=time.time() - start_time
            )
            
            # Cache successful results
            self.cache[cache_key] = result
            
            return result
            
        except Exception as e:
            return ToolResult(
                tool_name="tavily",
                success=False,
                result=None,
                error=str(e),
                execution_time=time.time() - start_time
            )
    
    def call_api(self, url: str, method: str = "GET", data: Dict = None) -> ToolResult:
        """Make API calls with error handling"""
        start_time = time.time()
        
        try:
            print(f"🌐 API: {method} request to {url}")
            
            with httpx.Client(timeout=10) as client:
                if method.upper() == "GET":
                    response = client.get(url)
                elif method.upper() == "POST":
                    response = client.post(url, json=data)
                else:
                    raise ValueError(f"Unsupported method: {method}")
                
                response.raise_for_status()
                
                return ToolResult(
                    tool_name="api_call",
                    success=True,
                    result=response.json() if response.content else None,
                    execution_time=time.time() - start_time
                )
                
        except Exception as e:
            return ToolResult(
                tool_name="api_call",
                success=False,
                result=None,
                error=str(e),
                execution_time=time.time() - start_time
            )

# Initialize tool manager
tool_manager = ToolManager()

# Research worker with tool integration
def research_worker(state: HierarchicalState) -> Command:
    """Research worker that uses Tavily for web search"""
    print("🔬 Research Worker: Conducting research with tools")
    
    if state.current_task:
        task = state.current_task
        
        # Extract search queries from task description
        query_extraction_prompt = f"""
        Extract 2-3 specific search queries from this research task:
        Task: {task.title}
        Description: {task.description}
        
        Return only the search queries, one per line, without any additional text.
        """
        
        input_tokens = count_tokens(query_extraction_prompt, "gpt-3.5-turbo")
        response = fast_llm.invoke([HumanMessage(content=query_extraction_prompt)])
        output_tokens = count_tokens(response.content, "gpt-3.5-turbo")
        
        total_tokens = input_tokens + output_tokens
        state.total_tokens_used += total_tokens
        state.estimated_cost += estimate_cost(total_tokens, "gpt-3.5-turbo")
        
        # Extract queries
        queries = [q.strip() for q in response.content.split('\n') if q.strip()]
        
        # Perform web searches
        search_results = []
        for query in queries[:3]:  # Limit to 3 queries
            result = tool_manager.search_web(query, max_results=3)
            if result.success:
                search_results.extend(result.result)
            else:
                print(f"⚠️ Search failed for '{query}': {result.error}")
        
        # Synthesize research findings
        if search_results:
            research_content = "\n\n".join([
                f"Title: {r['title']}\nURL: {r['url']}\nContent: {r['content']}"
                for r in search_results[:5]  # Limit to top 5 results
            ])
            
            synthesis_prompt = f"""
            You are a research specialist. Synthesize these web search results for the task:
            
            Task: {task.title}
            Description: {task.description}
            
            Search Results:
            {research_content[:3000]}  # Limit content to manage tokens
            
            Provide a comprehensive research summary with key findings, insights, and sources.
            """
            
            input_tokens = count_tokens(synthesis_prompt, "gpt-3.5-turbo")
            synthesis_response = worker_llm.invoke([HumanMessage(content=synthesis_prompt)])
            output_tokens = count_tokens(synthesis_response.content, "gpt-3.5-turbo")
            
            total_tokens += input_tokens + output_tokens
            state.total_tokens_used += total_tokens
            state.estimated_cost += estimate_cost(total_tokens, "gpt-3.5-turbo")
            
            task.result = synthesis_response.content
            task.actual_tokens = total_tokens
            
        else:
            # Fallback to knowledge-based research
            fallback_prompt = f"""
            Conduct research on: {task.title}
            Description: {task.description}
            
            Provide comprehensive research findings based on your knowledge.
            """
            
            fallback_response = worker_llm.invoke([HumanMessage(content=fallback_prompt)])
            task.result = fallback_response.content
        
        task.status = "completed"
        task.completed_at = datetime.now().isoformat()
        
        # Move to completed tasks
        state.completed_tasks.append(task)
        state.tasks = [t for t in state.tasks if t.task_id != task.task_id]
        state.current_task = None
        
        print(f"✅ Research completed (Tokens: {task.actual_tokens}, Cost: ${state.estimated_cost:.4f})")
    
    return Command(goto="worker_coordinator", update=state.model_dump())

# Analysis worker
def analysis_worker(state: HierarchicalState) -> Command:
    """Analysis worker for data processing and insights"""
    print("📊 Analysis Worker: Processing and analyzing data")
    
    if state.current_task:
        task = state.current_task
        
        # Get data from completed research tasks
        research_data = "\n\n".join([
            f"Research: {t.title}\nFindings: {t.result}"
            for t in state.completed_tasks
            if t.task_type == TaskType.RESEARCH and t.result
        ])
        
        analysis_prompt = f"""
        You are a data analysis specialist. Analyze the following for the task:
        
        Task: {task.title}
        Description: {task.description}
        
        Available Research Data:
        {research_data[:2000] if research_data else 'No research data available'}
        
        Provide detailed analysis with:
        1. Key patterns and trends
        2. Statistical insights
        3. Comparative analysis
        4. Actionable recommendations
        5. Risk assessment
        """
        
        input_tokens = count_tokens(analysis_prompt, "gpt-3.5-turbo")
        response = worker_llm.invoke([HumanMessage(content=analysis_prompt)])
        output_tokens = count_tokens(response.content, "gpt-3.5-turbo")
        
        total_tokens = input_tokens + output_tokens
        state.total_tokens_used += total_tokens
        state.estimated_cost += estimate_cost(total_tokens, "gpt-3.5-turbo")
        
        task.result = response.content
        task.actual_tokens = total_tokens
        task.status = "completed"
        task.completed_at = datetime.now().isoformat()
        
        # Move to completed tasks
        state.completed_tasks.append(task)
        state.tasks = [t for t in state.tasks if t.task_id != task.task_id]
        state.current_task = None
        
        print(f"✅ Analysis completed (Tokens: {total_tokens}, Cost: ${state.estimated_cost:.4f})")
    
    return Command(goto="worker_coordinator", update=state.model_dump())

print("🛠️ Tool-integrated workers defined")

### 3. Parallel Execution and Map-Reduce Pattern

In [ ]:
class ParallelTask(BaseModel):
    """Task for parallel execution"""
    task_id: str
    data_chunk: Dict[str, Any]
    operation: str
    result: Optional[Dict[str, Any]] = None
    status: str = "pending"
    worker_id: Optional[str] = None
    execution_time: Optional[float] = None
    tokens_used: int = 0

class MapReduceState(BaseModel):
    """State for map-reduce operations"""
    messages: List[BaseMessage] = Field(default_factory=list)
    input_data: List[Dict[str, Any]] = Field(default_factory=list)
    parallel_tasks: List[ParallelTask] = Field(default_factory=list)
    completed_tasks: List[ParallelTask] = Field(default_factory=list)
    map_results: List[Dict[str, Any]] = Field(default_factory=list)
    reduce_result: Optional[Dict[str, Any]] = None
    operation_type: str = "analysis"
    total_tokens_used: int = 0
    estimated_cost: float = 0.0
    performance_metrics: Dict[str, Any] = Field(default_factory=dict)

def map_coordinator(state: MapReduceState) -> Command:
    """Coordinates the map phase of parallel processing"""
    print("🗺️ Map Coordinator: Distributing tasks for parallel processing")
    
    # Create parallel tasks from input data
    if state.input_data and not state.parallel_tasks:
        for i, data_chunk in enumerate(state.input_data):
            task = ParallelTask(
                task_id=f"map_task_{i}",
                data_chunk=data_chunk,
                operation=state.operation_type,
                worker_id=f"worker_{i % 3}"  # Distribute among 3 workers
            )
            state.parallel_tasks.append(task)
        
        print(f"📊 Created {len(state.parallel_tasks)} parallel tasks")
    
    # Process tasks (simulating parallel execution)
    pending_tasks = [t for t in state.parallel_tasks if t.status == "pending"]
    
    if pending_tasks:
        # Take first pending task
        current_task = pending_tasks[0]
        current_task.status = "processing"
        
        # Route to appropriate worker based on operation type
        if state.operation_type == "analysis":
            return Command(goto="map_analyzer", update=state.model_dump())
        elif state.operation_type == "summarization":
            return Command(goto="map_summarizer", update=state.model_dump())
        else:
            return Command(goto="map_processor", update=state.model_dump())
    
    # All tasks completed, move to reduce phase
    return Command(goto="reduce_aggregator", update=state.model_dump())

def map_analyzer(state: MapReduceState) -> Command:
    """Processes individual data chunks in parallel"""
    print("🔍 Map Analyzer: Processing data chunk")
    
    # Find current processing task
    current_task = next((t for t in state.parallel_tasks if t.status == "processing"), None)
    
    if current_task:
        start_time = time.time()
        
        # Analyze the data chunk
        data_content = json.dumps(current_task.data_chunk, indent=2)
        
        analysis_prompt = f"""
        You are processing a data chunk in a parallel analysis system.
        
        Data chunk to analyze:
        {data_content[:1000]}  # Limit content size
        
        Provide analysis with:
        1. Key metrics and statistics
        2. Notable patterns or anomalies
        3. Summary insights
        4. Quality score (1-10)
        
        Format as JSON with keys: metrics, patterns, insights, quality_score
        """
        
        input_tokens = count_tokens(analysis_prompt, "gpt-3.5-turbo")
        response = worker_llm.invoke([HumanMessage(content=analysis_prompt)])
        output_tokens = count_tokens(response.content, "gpt-3.5-turbo")
        
        total_tokens = input_tokens + output_tokens
        current_task.tokens_used = total_tokens
        state.total_tokens_used += total_tokens
        state.estimated_cost += estimate_cost(total_tokens, "gpt-3.5-turbo")
        
        # Parse result (with error handling)
        try:
            result_data = json.loads(response.content)
        except json.JSONDecodeError:
            result_data = {
                "metrics": "Error parsing response",
                "patterns": response.content[:200],
                "insights": "Failed to parse structured output",
                "quality_score": 5
            }
        
        current_task.result = result_data
        current_task.status = "completed"
        current_task.execution_time = time.time() - start_time
        
        # Move to completed tasks
        state.completed_tasks.append(current_task)
        state.map_results.append(result_data)
        
        print(f"✅ Chunk analyzed in {current_task.execution_time:.2f}s (Tokens: {total_tokens})")
    
    # Continue with next task
    return Command(goto="map_coordinator", update=state.model_dump())

def reduce_aggregator(state: MapReduceState) -> Command:
    """Aggregates results from parallel processing"""
    print("🔄 Reduce Aggregator: Combining parallel results")
    
    if state.map_results:
        # Aggregate all map results
        aggregation_prompt = f"""
        You are aggregating results from parallel data processing.
        
        Map Results Summary:
        - Total chunks processed: {len(state.map_results)}
        - Average quality score: {sum(r.get('quality_score', 5) for r in state.map_results) / len(state.map_results):.2f}
        
        Individual Results:
        {json.dumps(state.map_results[:5], indent=2)[:2000]}  # Show first 5 results
        
        Provide a comprehensive aggregated analysis with:
        1. Overall patterns across all chunks
        2. Combined metrics and statistics
        3. Key insights and recommendations
        4. Quality assessment
        5. Performance summary
        
        Format as JSON with appropriate structure.
        """
        
        input_tokens = count_tokens(aggregation_prompt, "gpt-4")
        response = manager_llm.invoke([HumanMessage(content=aggregation_prompt)])
        output_tokens = count_tokens(response.content, "gpt-4")
        
        total_tokens = input_tokens + output_tokens
        state.total_tokens_used += total_tokens
        state.estimated_cost += estimate_cost(total_tokens, "gpt-4")
        
        # Parse and store final result
        try:
            state.reduce_result = json.loads(response.content)
        except json.JSONDecodeError:
            state.reduce_result = {
                "summary": response.content,
                "total_chunks": len(state.map_results),
                "aggregation_method": "text_summary"
            }
        
        # Calculate performance metrics
        total_execution_time = sum(t.execution_time or 0 for t in state.completed_tasks)
        avg_execution_time = total_execution_time / len(state.completed_tasks) if state.completed_tasks else 0
        
        state.performance_metrics = {
            "total_tasks": len(state.completed_tasks),
            "total_execution_time": total_execution_time,
            "average_task_time": avg_execution_time,
            "total_tokens": state.total_tokens_used,
            "estimated_cost": state.estimated_cost,
            "cost_per_task": state.estimated_cost / len(state.completed_tasks) if state.completed_tasks else 0
        }
        
        state.messages.append(response)
        
        print(f"🎯 Aggregation complete: {len(state.map_results)} results combined")
        print(f"📊 Performance: {total_execution_time:.2f}s total, ${state.estimated_cost:.4f} cost")
    
    return Command(finish=state.model_dump())

print("⚡ Map-reduce pattern implemented")

### 4. Performance and Cost Optimization

In [ ]:
class OptimizationConfig(BaseModel):
    """Configuration for performance optimization"""
    max_tokens_per_request: int = 1000
    use_gpt4_for_complex_only: bool = True
    enable_caching: bool = True
    enable_batching: bool = True
    max_batch_size: int = 5
    cost_threshold: float = 1.0  # Stop if cost exceeds this
    token_threshold: int = 10000  # Switch to cheaper model

class OptimizedState(BaseModel):
    """State with optimization tracking"""
    messages: List[BaseMessage] = Field(default_factory=list)
    tasks: List[Task] = Field(default_factory=list)
    completed_tasks: List[Task] = Field(default_factory=list)
    optimization_config: OptimizationConfig = Field(default_factory=OptimizationConfig)
    cost_tracking: Dict[str, float] = Field(default_factory=dict)
    token_tracking: Dict[str, int] = Field(default_factory=dict)
    model_usage: Dict[str, int] = Field(default_factory=dict)
    cache_hits: int = 0
    cache_misses: int = 0
    optimization_decisions: List[str] = Field(default_factory=list)
    performance_metrics: Dict[str, Any] = Field(default_factory=dict)

class OptimizedAgent:
    """Agent with performance and cost optimization"""
    
    def __init__(self, config: OptimizationConfig):
        self.config = config
        self.response_cache = {}
        self.model_selector = ModelSelector()
    
    def select_optimal_model(self, task_complexity: str, current_cost: float) -> ChatOpenAI:
        """Select the most cost-effective model for the task"""
        
        # Cost-based selection
        if current_cost > self.config.cost_threshold:
            return fast_llm  # Cheapest option
        
        # Complexity-based selection
        if task_complexity in ["simple", "routine"]:
            return worker_llm  # GPT-3.5-turbo
        elif task_complexity in ["moderate", "analysis"]:
            return manager_llm if self.config.use_gpt4_for_complex_only else worker_llm
        else:  # complex, strategic
            return executive_llm  # GPT-4
    
    def optimize_prompt(self, prompt: str, max_tokens: int) -> str:
        """Optimize prompt for token efficiency"""
        
        # If prompt is too long, summarize it
        current_tokens = count_tokens(prompt)
        
        if current_tokens > max_tokens:
            # Use fast model to summarize the prompt
            summary_prompt = f"""
            Summarize this prompt to under {max_tokens} tokens while preserving key instructions:
            
            {prompt[:2000]}...
            
            Keep all essential details and instructions.
            """
            
            try:
                response = fast_llm.invoke([HumanMessage(content=summary_prompt)])
                optimized_prompt = response.content
                
                # Verify it's actually shorter
                if count_tokens(optimized_prompt) < current_tokens:
                    return optimized_prompt
            except Exception:
                pass  # Fall back to truncation
            
            # Fallback: simple truncation
            return prompt[:max_tokens * 4]  # Rough token-to-char ratio
        
        return prompt
    
    def get_cached_response(self, prompt_hash: str) -> Optional[str]:
        """Get cached response if available"""
        if self.config.enable_caching:
            return self.response_cache.get(prompt_hash)
        return None
    
    def cache_response(self, prompt_hash: str, response: str):
        """Cache response for future use"""
        if self.config.enable_caching:
            self.response_cache[prompt_hash] = response

class ModelSelector:
    """Intelligent model selection based on task requirements"""
    
    def __init__(self):
        self.complexity_keywords = {
            "simple": ["summarize", "list", "extract", "format", "convert"],
            "moderate": ["analyze", "compare", "evaluate", "research", "process"],
            "complex": ["strategic", "design", "architect", "plan", "solve", "create"]
        }
    
    def assess_complexity(self, task_description: str) -> str:
        """Assess task complexity based on description"""
        task_lower = task_description.lower()
        
        complexity_scores = {"simple": 0, "moderate": 0, "complex": 0}
        
        for complexity, keywords in self.complexity_keywords.items():
            for keyword in keywords:
                if keyword in task_lower:
                    complexity_scores[complexity] += 1
        
        # Return the complexity with highest score
        return max(complexity_scores.items(), key=lambda x: x[1])[0]

def optimized_processor(state: OptimizedState) -> Command:
    """Optimized processor with cost and performance tracking"""
    print("⚡ Optimized Processor: Processing with performance optimization")
    
    agent = OptimizedAgent(state.optimization_config)
    current_cost = sum(state.cost_tracking.values())
    
    # Check cost threshold
    if current_cost > state.optimization_config.cost_threshold:
        state.optimization_decisions.append(f"Stopped processing: cost threshold ${state.optimization_config.cost_threshold} exceeded")
        return Command(finish=state.model_dump())
    
    pending_tasks = [t for t in state.tasks if t.status == "pending"]
    
    if pending_tasks:
        # Process tasks in batches for efficiency
        batch_size = min(state.optimization_config.max_batch_size, len(pending_tasks))
        batch_tasks = pending_tasks[:batch_size]
        
        for task in batch_tasks:
            start_time = time.time()
            
            # Assess task complexity
            complexity = agent.model_selector.assess_complexity(task.description)
            
            # Select optimal model
            selected_model = agent.select_optimal_model(complexity, current_cost)
            model_name = "gpt-4" if selected_model == executive_llm or selected_model == manager_llm else "gpt-3.5-turbo"
            
            # Track model usage
            state.model_usage[model_name] = state.model_usage.get(model_name, 0) + 1
            
            # Create optimized prompt
            base_prompt = f"""
            Task: {task.title}
            Description: {task.description}
            Complexity: {complexity}
            
            Provide efficient, focused output for this {complexity} task.
            """
            
            optimized_prompt = agent.optimize_prompt(
                base_prompt, 
                state.optimization_config.max_tokens_per_request
            )
            
            # Check cache first
            import hashlib
            prompt_hash = hashlib.md5(optimized_prompt.encode()).hexdigest()
            cached_response = agent.get_cached_response(prompt_hash)
            
            if cached_response:
                response_content = cached_response
                state.cache_hits += 1
                tokens_used = count_tokens(cached_response, model_name)
                state.optimization_decisions.append(f"Used cached response for task {task.task_id}")
            else:
                # Make API call
                state.cache_misses += 1
                
                input_tokens = count_tokens(optimized_prompt, model_name)
                response = selected_model.invoke([HumanMessage(content=optimized_prompt)])
                output_tokens = count_tokens(response.content, model_name)
                tokens_used = input_tokens + output_tokens
                
                response_content = response.content
                
                # Cache the response
                agent.cache_response(prompt_hash, response_content)
            
            # Update tracking
            cost_for_task = estimate_cost(tokens_used, model_name)
            state.cost_tracking[task.task_id] = cost_for_task
            state.token_tracking[task.task_id] = tokens_used
            
            # Complete task
            task.result = response_content
            task.status = "completed"
            task.completed_at = datetime.now().isoformat()
            task.actual_tokens = tokens_used
            
            execution_time = time.time() - start_time
            
            state.optimization_decisions.append(
                f"Task {task.task_id}: {model_name}, {tokens_used} tokens, ${cost_for_task:.4f}, {execution_time:.2f}s"
            )
            
            # Move to completed
            state.completed_tasks.append(task)
        
        # Remove processed tasks
        state.tasks = [t for t in state.tasks if t.task_id not in [bt.task_id for bt in batch_tasks]]
        
        print(f"✅ Processed batch of {len(batch_tasks)} tasks")
    
    # Calculate final performance metrics
    total_cost = sum(state.cost_tracking.values())
    total_tokens = sum(state.token_tracking.values())
    cache_hit_rate = state.cache_hits / (state.cache_hits + state.cache_misses) if (state.cache_hits + state.cache_misses) > 0 else 0
    
    state.performance_metrics = {
        "total_cost": total_cost,
        "total_tokens": total_tokens,
        "completed_tasks": len(state.completed_tasks),
        "cache_hit_rate": cache_hit_rate,
        "model_usage": dict(state.model_usage),
        "avg_cost_per_task": total_cost / len(state.completed_tasks) if state.completed_tasks else 0,
        "avg_tokens_per_task": total_tokens / len(state.completed_tasks) if state.completed_tasks else 0
    }
    
    if state.tasks:  # More tasks to process
        return Command(goto="optimized_processor", update=state.model_dump())
    else:
        return Command(finish=state.model_dump())

print("🎯 Performance optimization components ready")

### 5. Building and Testing Complete Systems

In [ ]:
# Build hierarchical system
def create_hierarchical_system():
    """Create complete hierarchical multi-agent system"""
    
    graph = StateGraph(HierarchicalState)
    
    # Add all agent levels
    graph.add_node("executive", executive_agent)
    graph.add_node("manager", manager_agent)
    graph.add_node("worker_coordinator", worker_coordinator)
    graph.add_node("research_worker", research_worker)
    graph.add_node("analysis_worker", analysis_worker)
    
    # Add a general worker for other tasks
    def general_worker(state: HierarchicalState) -> Command:
        if state.current_task:
            task = state.current_task
            response = worker_llm.invoke([HumanMessage(content=f"Complete this task: {task.description}")])
            task.result = response.content
            task.status = "completed"
            task.completed_at = datetime.now().isoformat()
            state.completed_tasks.append(task)
            state.tasks = [t for t in state.tasks if t.task_id != task.task_id]
            state.current_task = None
        return Command(goto="worker_coordinator", update=state.model_dump())
    
    graph.add_node("general_worker", general_worker)
    
    # Results aggregator
    def results_aggregator(state: HierarchicalState) -> Command:
        print("📊 Results Aggregator: Compiling final results")
        
        if state.completed_tasks:
            results_summary = "\n\n".join([
                f"Task: {t.title}\nResult: {t.result[:200]}..."
                for t in state.completed_tasks
            ])
            
            final_prompt = f"""
            Compile a comprehensive executive summary from these completed tasks:
            
            Strategic Plan:
            {state.strategic_plan.get('plan', 'N/A')[:300]}...
            
            Operational Plan:
            {state.operational_plan.get('plan', 'N/A')[:300]}...
            
            Completed Tasks:
            {results_summary[:1500]}...
            
            Provide executive summary with key achievements, insights, and recommendations.
            """
            
            response = executive_llm.invoke([HumanMessage(content=final_prompt)])
            state.messages.append(response)
            
            print(f"✅ Executive summary generated")
        
        return Command(finish=state.model_dump())
    
    graph.add_node("results_aggregator", results_aggregator)
    
    # Connect the flow
    graph.add_edge(START, "executive")
    
    # Compile with persistence
    saver = SqliteSaver.from_conn_string("hierarchical_system.db")
    return graph.compile(checkpointer=saver)

# Build map-reduce system
def create_mapreduce_system():
    """Create map-reduce parallel processing system"""
    
    graph = StateGraph(MapReduceState)
    
    graph.add_node("map_coordinator", map_coordinator)
    graph.add_node("map_analyzer", map_analyzer)
    graph.add_node("reduce_aggregator", reduce_aggregator)
    
    # Add summarizer for different operations
    def map_summarizer(state: MapReduceState) -> Command:
        current_task = next((t for t in state.parallel_tasks if t.status == "processing"), None)
        if current_task:
            data_content = json.dumps(current_task.data_chunk, indent=2)
            response = fast_llm.invoke([HumanMessage(content=f"Summarize this data: {data_content[:500]}")])
            current_task.result = {"summary": response.content}
            current_task.status = "completed"
            state.completed_tasks.append(current_task)
            state.map_results.append(current_task.result)
        return Command(goto="map_coordinator", update=state.model_dump())
    
    def map_processor(state: MapReduceState) -> Command:
        current_task = next((t for t in state.parallel_tasks if t.status == "processing"), None)
        if current_task:
            data_content = json.dumps(current_task.data_chunk, indent=2)
            response = worker_llm.invoke([HumanMessage(content=f"Process this data: {data_content[:500]}")])
            current_task.result = {"processed_output": response.content}
            current_task.status = "completed"
            state.completed_tasks.append(current_task)
            state.map_results.append(current_task.result)
        return Command(goto="map_coordinator", update=state.model_dump())
    
    graph.add_node("map_summarizer", map_summarizer)
    graph.add_node("map_processor", map_processor)
    
    graph.add_edge(START, "map_coordinator")
    
    saver = SqliteSaver.from_conn_string("mapreduce_system.db")
    return graph.compile(checkpointer=saver)

# Build optimized system
def create_optimized_system():
    """Create cost-optimized processing system"""
    
    graph = StateGraph(OptimizedState)
    graph.add_node("optimized_processor", optimized_processor)
    graph.add_edge(START, "optimized_processor")
    
    saver = SqliteSaver.from_conn_string("optimized_system.db")
    return graph.compile(checkpointer=saver)

print("🏗️ All advanced systems created")

### 6. Testing the Complete Advanced System

In [ ]:
# Test Hierarchical System
print("🧪 Testing Hierarchical Multi-Agent System...")

hierarchical_app = create_hierarchical_system()

# Create test tasks for hierarchical system
test_tasks = [
    Task(
        task_id="task_001",
        title="Market Research on AI Tools",
        description="Research the current market for AI development tools and frameworks",
        task_type=TaskType.RESEARCH,
        priority=1,
        assigned_level=AgentLevel.WORKER
    ),
    Task(
        task_id="task_002",
        title="Competitive Analysis",
        description="Analyze competitors in the AI agent development space",
        task_type=TaskType.ANALYSIS,
        priority=2,
        assigned_level=AgentLevel.WORKER
    )
]

hierarchical_state = HierarchicalState(tasks=test_tasks)
hierarchical_config = {"configurable": {"thread_id": "hierarchical-test"}}

print("\n🚀 Running hierarchical system...")
hierarchical_result = hierarchical_app.invoke(hierarchical_state, config=hierarchical_config)

print(f"\n📊 Hierarchical System Results:")
print(f"✅ Completed Tasks: {len(hierarchical_result.completed_tasks)}")
print(f"💰 Total Cost: ${hierarchical_result.estimated_cost:.4f}")
print(f"🔢 Total Tokens: {hierarchical_result.total_tokens_used}")
print(f"📋 Strategic Plan: {'✅ Created' if hierarchical_result.strategic_plan else '❌ Missing'}")
print(f"📋 Operational Plan: {'✅ Created' if hierarchical_result.operational_plan else '❌ Missing'}")

In [ ]:
# Test Map-Reduce System
print("\n🧪 Testing Map-Reduce Parallel Processing...")

mapreduce_app = create_mapreduce_system()

# Create test data for parallel processing
test_data = [
    {"id": 1, "category": "technology", "value": 100, "description": "AI development tools"},
    {"id": 2, "category": "technology", "value": 150, "description": "Machine learning frameworks"},
    {"id": 3, "category": "business", "value": 200, "description": "Market analysis data"},
    {"id": 4, "category": "business", "value": 120, "description": "Customer feedback"},
    {"id": 5, "category": "technology", "value": 180, "description": "Performance metrics"}
]

mapreduce_state = MapReduceState(
    input_data=test_data,
    operation_type="analysis"
)
mapreduce_config = {"configurable": {"thread_id": "mapreduce-test"}}

print("\n🚀 Running map-reduce system...")
mapreduce_result = mapreduce_app.invoke(mapreduce_state, config=mapreduce_config)

print(f"\n📊 Map-Reduce Results:")
print(f"⚡ Parallel Tasks: {len(mapreduce_result.completed_tasks)}")
print(f"📈 Map Results: {len(mapreduce_result.map_results)}")
print(f"🎯 Reduce Result: {'✅ Generated' if mapreduce_result.reduce_result else '❌ Missing'}")
print(f"💰 Total Cost: ${mapreduce_result.estimated_cost:.4f}")
print(f"⏱️ Performance Metrics:")
for key, value in mapreduce_result.performance_metrics.items():
    print(f"   {key}: {value}")

In [ ]:
# Test Optimized System
print("\n🧪 Testing Cost-Optimized Processing...")

optimized_app = create_optimized_system()

# Create tasks with different complexity levels
optimization_tasks = [
    Task(
        task_id="opt_001",
        title="Simple Data Extraction",
        description="Extract key metrics from a simple dataset",
        task_type=TaskType.EXECUTION,
        priority=3,
        assigned_level=AgentLevel.WORKER
    ),
    Task(
        task_id="opt_002",
        title="Strategic Planning",
        description="Develop a comprehensive strategic plan for AI product development",
        task_type=TaskType.STRATEGIC,
        priority=1,
        assigned_level=AgentLevel.EXECUTIVE
    ),
    Task(
        task_id="opt_003",
        title="Data Analysis",
        description="Analyze customer behavior patterns and trends",
        task_type=TaskType.ANALYSIS,
        priority=2,
        assigned_level=AgentLevel.WORKER
    )
]

optimization_config = OptimizationConfig(
    max_tokens_per_request=500,
    use_gpt4_for_complex_only=True,
    enable_caching=True,
    cost_threshold=0.50
)

optimized_state = OptimizedState(
    tasks=optimization_tasks,
    optimization_config=optimization_config
)
optimized_config = {"configurable": {"thread_id": "optimized-test"}}

print("\n🚀 Running optimized system...")
optimized_result = optimized_app.invoke(optimized_state, config=optimized_config)

print(f"\n📊 Optimization Results:")
print(f"✅ Completed Tasks: {len(optimized_result.completed_tasks)}")
print(f"💰 Total Cost: ${optimized_result.performance_metrics.get('total_cost', 0):.4f}")
print(f"🔢 Total Tokens: {optimized_result.performance_metrics.get('total_tokens', 0)}")
print(f"🎯 Cache Hit Rate: {optimized_result.performance_metrics.get('cache_hit_rate', 0):.2%}")
print(f"📱 Model Usage: {optimized_result.performance_metrics.get('model_usage', {})}")
print(f"💡 Optimization Decisions:")
for decision in optimized_result.optimization_decisions[-3:]:  # Show last 3 decisions
    print(f"   {decision}")

print("\n🎉 All advanced systems tested successfully!")
print(f"\n📈 Total Systems Performance Summary:")
print(f"🏗️ Hierarchical: ${hierarchical_result.estimated_cost:.4f} cost, {hierarchical_result.total_tokens_used} tokens")
print(f"⚡ Map-Reduce: ${mapreduce_result.estimated_cost:.4f} cost, {mapreduce_result.total_tokens_used} tokens")
print(f"🎯 Optimized: ${optimized_result.performance_metrics.get('total_cost', 0):.4f} cost, {optimized_result.performance_metrics.get('total_tokens', 0)} tokens")

---
## 🛠️ Practical Exercises (30 minutes)

### Exercise 1: Build a Content Production Pipeline
**Goal**: Create a hierarchical system for automated content production.

**Requirements**:
- Content strategist (executive level) for content planning
- Content manager (manager level) for workflow coordination
- Research writers, editors, and publishers (worker level)
- Integrate Tavily for research and fact-checking
- Implement cost optimization for different content types
- Include quality control and approval workflows

In [ ]:
# Exercise 1: Your implementation here
class ContentPiece(BaseModel):
    """Content piece for production pipeline"""
    # TODO: Define your content structure
    pass

class ContentProductionState(BaseModel):
    """State for content production pipeline"""
    # TODO: Define your content production state
    pass

def content_strategist_agent(state: ContentProductionState) -> Command:
    """Develops content strategy and editorial calendar"""
    # TODO: Implement content strategy logic
    pass

def research_writer_agent(state: ContentProductionState) -> Command:
    """Researches and writes content using tools"""
    # TODO: Implement research and writing logic with Tavily integration
    pass

# TODO: Create and test your content production pipeline
print("📝 Exercise 1: Implement your content production pipeline here")

### Exercise 2: Implement a Real-time Data Processing System
**Goal**: Build a map-reduce system for processing large datasets in real-time.

**Requirements**:
- Data ingestion and preprocessing workers
- Parallel analysis workers for different data types
- Real-time aggregation and reporting
- API integration for external data sources
- Performance monitoring and auto-scaling
- Error handling and data quality checks

In [ ]:
# Exercise 2: Your implementation here
class DataStream(BaseModel):
    """Data stream for real-time processing"""
    # TODO: Define your data stream structure
    pass

class RealTimeProcessingState(BaseModel):
    """State for real-time data processing"""
    # TODO: Define your real-time processing state
    pass

def data_ingestion_worker(state: RealTimeProcessingState) -> Command:
    """Ingests data from various sources"""
    # TODO: Implement data ingestion logic
    pass

def parallel_analyzer_worker(state: RealTimeProcessingState) -> Command:
    """Analyzes data chunks in parallel"""
    # TODO: Implement parallel analysis logic
    pass

# TODO: Create and test your real-time processing system
print("📊 Exercise 2: Implement your real-time data processing system here")

### Challenge: Create an AI-Powered Business Intelligence Platform
**Goal**: Build a comprehensive BI platform with advanced AI capabilities.

**Advanced Requirements**:
- Executive dashboard with strategic insights
- Automated report generation and distribution
- Predictive analytics and forecasting
- Natural language query interface
- Multi-source data integration (APIs, databases, web scraping)
- Cost-optimized processing for different report types
- Real-time alerts and anomaly detection
- Role-based access and personalized insights

In [ ]:
# Challenge: Your implementation here
class BusinessInsight(BaseModel):
    """Business insight structure"""
    # TODO: Design comprehensive business insight schema
    pass

class BIplatformState(BaseModel):
    """State for business intelligence platform"""
    # TODO: Design complex BI platform state
    pass

# TODO: Implement your AI-powered business intelligence platform
print("🎯 Challenge: Build your AI-powered business intelligence platform here")
print("💡 Hint: Consider data sources, analysis types, user roles, and cost optimization")

---
## 📚 Solutions and Best Practices

### Exercise 1 Solution: Content Production Pipeline

In [ ]:
# Complete solution for Exercise 1
from enum import Enum

class ContentType(str, Enum):
    BLOG_POST = "blog_post"
    ARTICLE = "article"
    SOCIAL_MEDIA = "social_media"
    NEWSLETTER = "newsletter"
    WHITE_PAPER = "white_paper"

class ContentStatus(str, Enum):
    PLANNED = "planned"
    RESEARCHING = "researching"
    WRITING = "writing"
    EDITING = "editing"
    REVIEW = "review"
    APPROVED = "approved"
    PUBLISHED = "published"

class ContentPiece(BaseModel):
    content_id: str
    title: str
    content_type: ContentType
    target_audience: str
    keywords: List[str] = Field(default_factory=list)
    research_sources: List[str] = Field(default_factory=list)
    content_brief: str
    draft_content: Optional[str] = None
    final_content: Optional[str] = None
    status: ContentStatus = ContentStatus.PLANNED
    assigned_writer: Optional[str] = None
    quality_score: Optional[float] = None
    word_count: int = 0
    estimated_tokens: int = 0
    actual_tokens: int = 0
    created_at: str = Field(default_factory=lambda: datetime.now().isoformat())

class ContentProductionState(BaseModel):
    messages: List[BaseMessage] = Field(default_factory=list)
    content_calendar: List[ContentPiece] = Field(default_factory=list)
    current_content: Optional[ContentPiece] = None
    editorial_guidelines: Dict[str, Any] = Field(default_factory=dict)
    quality_standards: Dict[str, Any] = Field(default_factory=dict)
    production_metrics: Dict[str, Any] = Field(default_factory=dict)
    total_tokens_used: int = 0
    estimated_cost: float = 0.0

def content_strategist_agent(state: ContentProductionState) -> Command:
    """Develops content strategy and editorial calendar"""
    print("📋 Content Strategist: Planning content strategy")
    
    # Create content strategy
    strategy_prompt = f"""
    You are a content strategist. Plan a content calendar for the next month.
    
    Current content queue: {len(state.content_calendar)} pieces
    
    Create a strategic plan including:
    1. Content themes and topics
    2. Content type distribution
    3. Target audience segments
    4. SEO keyword strategy
    5. Publishing schedule
    
    Provide actionable content briefs for upcoming pieces.
    """
    
    response = executive_llm.invoke([HumanMessage(content=strategy_prompt)])
    
    # Store editorial guidelines
    state.editorial_guidelines = {
        "strategy": response.content,
        "created_at": datetime.now().isoformat(),
        "agent": "content_strategist"
    }
    
    state.messages.append(response)
    
    return Command(goto="content_manager", update=state.model_dump())

def research_writer_agent(state: ContentProductionState) -> Command:
    """Researches and writes content using tools"""
    print("✍️ Research Writer: Creating content with research")
    
    if state.current_content:
        content = state.current_content
        
        # Research phase using Tavily
        research_results = []
        for keyword in content.keywords[:3]:  # Limit research
            result = tool_manager.search_web(f"{keyword} {content.target_audience}", max_results=2)
            if result.success:
                research_results.extend(result.result)
        
        # Writing phase
        research_summary = "\n".join([
            f"- {r['title']}: {r['content'][:100]}..."
            for r in research_results[:5]
        ])
        
        writing_prompt = f"""
        Write a {content.content_type.value} with the following specifications:
        
        Title: {content.title}
        Brief: {content.content_brief}
        Target Audience: {content.target_audience}
        Keywords to include: {', '.join(content.keywords)}
        
        Research Sources:
        {research_summary}
        
        Create engaging, well-structured content that meets editorial standards.
        """
        
        # Select model based on content type complexity
        model = manager_llm if content.content_type in [ContentType.WHITE_PAPER, ContentType.ARTICLE] else worker_llm
        model_name = "gpt-4" if model == manager_llm else "gpt-3.5-turbo"
        
        response = model.invoke([HumanMessage(content=writing_prompt)])
        
        tokens_used = count_tokens(writing_prompt + response.content, model_name)
        state.total_tokens_used += tokens_used
        state.estimated_cost += estimate_cost(tokens_used, model_name)
        
        # Update content
        content.draft_content = response.content
        content.word_count = len(response.content.split())
        content.actual_tokens = tokens_used
        content.status = ContentStatus.EDITING
        content.assigned_writer = "research_writer"
        
        print(f"✅ Content drafted: {content.word_count} words, {tokens_used} tokens")
    
    return Command(goto="content_editor", update=state.model_dump())

def content_editor_agent(state: ContentProductionState) -> Command:
    """Edits and improves content quality"""
    print("📝 Content Editor: Reviewing and editing content")
    
    if state.current_content and state.current_content.draft_content:
        content = state.current_content
        
        editing_prompt = f"""
        You are a professional editor. Review and improve this {content.content_type.value}:
        
        Title: {content.title}
        Target Audience: {content.target_audience}
        
        Draft Content:
        {content.draft_content[:2000]}...
        
        Provide:
        1. Edited version with improvements
        2. Quality score (1-10)
        3. Key improvements made
        4. Recommendations for further enhancement
        
        Focus on clarity, engagement, and target audience fit.
        """
        
        response = manager_llm.invoke([HumanMessage(content=editing_prompt)])
        
        # Extract quality score (simplified)
        try:
            import re
            score_match = re.search(r'quality score[:\s]*(\d+)', response.content.lower())
            content.quality_score = float(score_match.group(1)) if score_match else 7.0
        except:
            content.quality_score = 7.0
        
        content.final_content = response.content
        content.status = ContentStatus.REVIEW
        
        print(f"✅ Content edited: Quality score {content.quality_score}/10")
    
    return Command(goto="content_manager", update=state.model_dump())

def content_manager_agent(state: ContentProductionState) -> Command:
    """Manages content workflow and quality control"""
    print("👔 Content Manager: Managing production workflow")
    
    # Check if there's content in review
    if state.current_content and state.current_content.status == ContentStatus.REVIEW:
        content = state.current_content
        
        # Approve if quality score is sufficient
        if content.quality_score and content.quality_score >= 7.0:
            content.status = ContentStatus.APPROVED
            print(f"✅ Content approved: {content.title}")
            
            # Move to completed calendar
            state.content_calendar.append(content)
            state.current_content = None
        else:
            print(f"⚠️ Content needs improvement: Quality score {content.quality_score}")
            content.status = ContentStatus.WRITING  # Send back for revision
    
    # Start new content if none in progress
    elif not state.current_content:
        # Create new content piece
        new_content = ContentPiece(
            content_id="content_001",
            title="The Future of AI in Business Operations",
            content_type=ContentType.BLOG_POST,
            target_audience="business executives",
            keywords=["artificial intelligence", "business automation", "digital transformation"],
            content_brief="Explore how AI is transforming business operations and what executives need to know"
        )
        
        state.current_content = new_content
        new_content.status = ContentStatus.RESEARCHING
        
        return Command(goto="research_writer", update=state.model_dump())
    
    # Calculate production metrics
    approved_content = [c for c in state.content_calendar if c.status == ContentStatus.APPROVED]
    
    state.production_metrics = {
        "total_pieces": len(state.content_calendar),
        "approved_pieces": len(approved_content),
        "average_quality": sum(c.quality_score for c in approved_content if c.quality_score) / len(approved_content) if approved_content else 0,
        "total_words": sum(c.word_count for c in approved_content),
        "total_cost": state.estimated_cost,
        "avg_cost_per_piece": state.estimated_cost / len(approved_content) if approved_content else 0
    }
    
    return Command(finish=state.model_dump())

# Create content production system
def create_content_production_system():
    graph = StateGraph(ContentProductionState)
    
    graph.add_node("content_strategist", content_strategist_agent)
    graph.add_node("content_manager", content_manager_agent)
    graph.add_node("research_writer", research_writer_agent)
    graph.add_node("content_editor", content_editor_agent)
    
    graph.add_edge(START, "content_strategist")
    
    saver = SqliteSaver.from_conn_string("content_production.db")
    return graph.compile(checkpointer=saver)

# Test content production system
content_app = create_content_production_system()
content_state = ContentProductionState()
content_config = {"configurable": {"thread_id": "content-production-test"}}

print("\n📝 Testing content production pipeline...")
content_result = content_app.invoke(content_state, config=content_config)

print(f"\n📊 Content Production Results:")
print(f"📋 Production Metrics:")
for key, value in content_result.production_metrics.items():
    print(f"   {key}: {value}")

if content_result.content_calendar:
    approved_content = [c for c in content_result.content_calendar if c.status == ContentStatus.APPROVED]
    for content in approved_content:
        print(f"\n✅ Approved Content: {content.title}")
        print(f"   Type: {content.content_type.value}, Words: {content.word_count}")
        print(f"   Quality: {content.quality_score}/10, Tokens: {content.actual_tokens}")

print("\n✅ Content production pipeline solution completed")

---
## 🔧 Troubleshooting Common Issues

### Tool Integration Issues
```python
# ❌ Common issue: Tool failures not handled
result = tavily_client.search(query)  # May fail

# ✅ Proper error handling
try:
    result = tool_manager.search_web(query)
    if not result.success:
        # Fallback to knowledge-based approach
        return fallback_processing(query)
except Exception as e:
    logger.error(f"Tool error: {e}")
    return error_response
```

### Performance Optimization Issues
```python
# ✅ Monitor token usage
def track_tokens(prompt: str, response: str, model: str):
    input_tokens = count_tokens(prompt, model)
    output_tokens = count_tokens(response, model)
    total_cost = estimate_cost(input_tokens + output_tokens, model)
    
    # Log for monitoring
    logger.info(f"Tokens: {input_tokens + output_tokens}, Cost: ${total_cost:.4f}")
```

### Parallel Processing Issues
```python
# ✅ Handle task failures in parallel processing
def safe_parallel_task(task_data):
    try:
        return process_task(task_data)
    except Exception as e:
        return {"error": str(e), "task_id": task_data.get("id")}
```

### Cost Control Issues
```python
# ✅ Implement cost controls
class CostController:
    def __init__(self, max_cost: float):
        self.max_cost = max_cost
        self.current_cost = 0.0
    
    def can_proceed(self, estimated_cost: float) -> bool:
        return (self.current_cost + estimated_cost) <= self.max_cost
```

---
## 📖 Summary and Course Completion

### What You've Mastered Across 5 Days:
✅ **Day 1**: LangGraph foundations, nodes, edges, and basic workflows  
✅ **Day 2**: State management, persistence, and error recovery  
✅ **Day 3**: Memory systems, knowledge management, and long-term retention  
✅ **Day 4**: Multi-agent communication, handoffs, and secure messaging  
✅ **Day 5**: Advanced architectures, tool integration, and optimization  

### Advanced Patterns You've Learned:
- **Hierarchical Systems**: Executive, manager, and worker agent layers
- **Parallel Processing**: Map-reduce patterns for scalable computation
- **Tool Integration**: Tavily search, API calls, and external services
- **Performance Optimization**: Model selection, caching, and cost control
- **Production Deployment**: Monitoring, error handling, and scalability

### Key Technical Skills:
- Multi-agent system architecture and design
- OpenAI API optimization and cost management
- Tool integration and error handling
- State management and persistence strategies
- Performance monitoring and optimization
- Production deployment best practices

### Production-Ready Capabilities:
- Build scalable multi-agent systems
- Implement cost-effective AI workflows
- Integrate external tools and APIs
- Handle errors and edge cases gracefully
- Monitor performance and optimize costs
- Deploy and maintain LangGraph applications

### Next Steps for Continued Learning:
1. **Explore LangGraph Cloud**: Deploy your systems to production
2. **Advanced Tool Integration**: Build custom tools and integrations
3. **Multi-Modal Agents**: Integrate vision and audio capabilities
4. **Enterprise Patterns**: Implement enterprise-grade security and compliance
5. **Community Contribution**: Share your patterns and learn from others

### Resources for Continued Growth:
- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [LangChain Academy](https://academy.langchain.com/)
- [OpenAI API Best Practices](https://platform.openai.com/docs/guides/production-best-practices)
- [LangGraph Community](https://github.com/langchain-ai/langgraph)

**🎉 Congratulations! You've completed the comprehensive 5-day LangGraph course and are now ready to build production-grade multi-agent AI systems!**

In [ ]:
# Course completion summary
print("🧹 Course complete! Final database files created:")
import os
db_files = [f for f in os.listdir('.') if f.endswith('.db')]
for db_file in db_files:
    if any(x in db_file for x in ['hierarchical', 'mapreduce', 'optimized', 'content']):
        print(f"  📁 {db_file}")

print("\n🎉 5-Day LangGraph Course Complete!")
print("🚀 You've mastered:")
print("   📚 Day 1: LangGraph Foundations")
print("   💾 Day 2: State Management & Persistence")
print("   🧠 Day 3: Memory Systems & Knowledge Management")
print("   🤝 Day 4: Multi-Agent Communication & Handoffs")
print("   🏗️ Day 5: Advanced Architectures & Tool Integration")
print("\n🎯 You're now ready to build production-grade multi-agent AI systems with LangGraph!")
print("🌟 Keep building amazing AI applications!")